In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import ants
import matplotlib.pyplot as plt

2022-12-01 17:10:59.309535: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 17:10:59.401907: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-01 17:10:59.919929: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-01 17:10:59.920029: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [6]:
CLASS_SUBFOLDERS = ['MCI/', 'AD/', 'CN/']
DB_SS_PATH = 'data/output/'

TFR_PATH = 'data/output/TFR/'

TFR_TRAIN = 'train.tfrecords'
TFR_TEST = 'test.tfrecords'
TFR_VAL = 'val.tfrecords'

train_tfrec = os.path.join(TFR_PATH, TFR_TRAIN)
test_tfrec = os.path.join(TFR_PATH, TFR_TEST)
val_tfrec = os.path.join(TFR_PATH, TFR_VAL)

LABELS = {'CN': 1, 'MCI': 0, 'AD': 0}
BINARY_LABELS = {'CN': 0, 'AD': 1}

TEST_SPLIT = 0.15
VALIDATION_SPLIT = 0.15

In [15]:
rm_dir = "data/output/AD"

for file in os.listdir(rm_dir):
  if ".mat" in os.path.join(rm_dir,file):
    os.remove(os.path.join(rm_dir,file))

n_AD = len(os.listdir(rm_dir))

In [16]:
rm_dir = "data/output/CN"

for file in os.listdir(rm_dir):
  if ".mat" in os.path.join(rm_dir,file):
    os.remove(os.path.join(rm_dir,file))

n_CN = len(os.listdir(rm_dir))

In [17]:
rm_dir = "data/output/MCI"

for file in os.listdir(rm_dir):
  if ".mat" in os.path.join(rm_dir,file):
    os.remove(os.path.join(rm_dir,file))

n_MCI = len(os.listdir(rm_dir))

In [12]:
# array for saving the filenames
filenames = np.array([])

# iterate all three class folders in the db
for subf in CLASS_SUBFOLDERS:
  # using the skull stripped data
  path = DB_SS_PATH + subf
  for name in os.listdir(path):
    complete_name = os.path.join(path, name)
    if os.path.isfile(complete_name):
        filenames = np.concatenate((filenames, complete_name), axis=None)
      

filenames.shape

(924,)

In [13]:
raw_dir = "data/raw_data"
for id in filenames:
    split_path = id.split('/')[-1]
    id_id = split_path.split('_')[0]
    for file in os.listdir(raw_dir):
        if id_id in os.path.join(raw_dir,file):
            os.remove(os.path.join(raw_dir,file))

In [14]:
for i in range(1000):
  np.random.shuffle(filenames)
  
test_margin = int(len(filenames) * TEST_SPLIT)
training_set, test_set = filenames[test_margin:], filenames[:test_margin]

validation_margin = int(len(training_set) * VALIDATION_SPLIT)
training_set, validation_set = training_set[validation_margin:], training_set[:validation_margin]

print('Training set:', training_set.shape)
print('Validation set:', validation_set.shape)
print('Test set:', test_set.shape)

Training set: (669,)
Validation set: (117,)
Test set: (138,)


In [8]:
description = pd.read_csv('data/data.csv')
description.head()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I63874,941_S_1202,CN,M,78,1,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,1/30/2007,NiFTI,NaN
1,I204843,941_S_1202,CN,M,81,8,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/14/2010,NiFTI,NaN
2,I137298,941_S_1202,CN,M,80,6,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,2/17/2009,NiFTI,NaN
3,I75150,941_S_1202,CN,M,78,3,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,8/24/2007,NiFTI,NaN
4,I105437,941_S_1202,CN,M,79,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,2/28/2008,NiFTI,NaN


In [9]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def load_image(abs_path):
    split_path = abs_path.split('/')
    label = LABELS[split_path[-2]]
    ants_image = ants.image_read(abs_path)
    img = ants_image.numpy()
    
    return img, label

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array

def create_tf_record(img_filenames, tf_rec_filename):
    writer = tf.io.TFRecordWriter(tf_rec_filename)
    for meta_data in img_filenames:
        img, label = load_image(meta_data)
        feature = {'label': _int64_feature(label),
                   'image': _bytes_feature(serialize_array(img))}
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
create_tf_record(training_set, train_tfrec)
create_tf_record(test_set, test_tfrec)
create_tf_record(validation_set, val_tfrec)

In [10]:
n_training_samples = 0
for record in tf.data.TFRecordDataset(train_tfrec):
  n_training_samples += 1
print ('Training samples:', n_training_samples)

2022-11-29 20:41:09.229378: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-29 20:41:09.393644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-29 20:41:09.394520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-29 20:41:09.395570: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other ope

Training samples: 1230


In [11]:
n_val_samples = 0
for record in tf.data.TFRecordDataset(val_tfrec):
  n_val_samples += 1
print ('Validation samples:', n_val_samples)

Validation samples: 217


In [12]:
n_test_samples = 0
for record in tf.data.TFRecordDataset(test_tfrec):
  n_test_samples += 1
print ('Test samples:', n_test_samples)

Test samples: 255


In [13]:
total_n_samples = n_test_samples + n_training_samples + n_val_samples
print ('Dataset size:', total_n_samples)

Dataset size: 1702


In [14]:
def _decode(serialized_example):
    IMG_SHAPE = (172, 220, 156, 1)
    feature = tf.io.parse_single_example(
        serialized_example,
        features = {
            'image': tf.io.FixedLenFeature([], tf.string),
            'label': tf.io.FixedLenFeature([], tf.int64)
        }
    )
    image = tf.io.parse_tensor(feature['image'], out_type=tf.float32)
    image = tf.reshape(image, IMG_SHAPE)

    label = tf.cast(feature['label'], tf.int32)
    #label = tf.one_hot(feature['label'], 3) #change to multiclass
    #label = tf.constant([1,0,0])[label] #change to binary

    return image, label

In [15]:
def read_dataset(epochs, batch_size, filename):
  dataset = tf.data.TFRecordDataset(filename)
  dataset = dataset.map(_decode, num_parallel_calls=10)
  dataset = dataset.shuffle(epochs * batch_size).repeat()
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(batch_size)
    
  return dataset

def read_dataset_for_test(epochs, batch_size, filename):
  dataset = tf.data.TFRecordDataset(filename)
  dataset = dataset.map(_decode, num_parallel_calls=10)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(batch_size)
    
  return dataset

In [ ]:
#########################
## OLD MODEL ARCHITECH ##
#########################

#from antspynet.architectures import create_resnet_model_3d
#from tensorflow.keras import Sequential, layers, models
#from tensorflow.keras import optimizers

#def set_nontrainable_layers(model):
#    model.trainable = False 
#    return model

#base_model = create_resnet_model_3d((172, 220, 156, 1))
#base_model = set_nontrainable_layers(base_model)
#flatten_layer = layers.Flatten()
#dense_layer = layers.Dense(512, activation='relu')
#prediction_layer = layers.Dense(3, activation='softmax')

#model = models.Sequential([
#        base_model,
#        flatten_layer,
#        dense_layer,
#        prediction_layer
#    ])
#
#model.summary()

In [16]:
n_nonCN = n_MCI + n_AD
n_total = n_nonCN + n_CN
initial_bias = np.log([n_CN/n_nonCN])
print(f"CN: {n_CN} - {n_CN/n_total}\nNon-CN: {n_nonCN} - {n_nonCN/n_total}\nTotal: {n_total}\nBias: {initial_bias}")

CN: 555 - 0.32608695652173914
Non-CN: 1147 - 0.6739130434782609
Total: 1702
Bias: [-0.725937]


In [18]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import optimizers

METRICS = [
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), #precision-recall curve
      tf.keras.metrics.BinaryAccuracy(name='accuracy')
]

initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True)

opt = optimizers.SGD(learning_rate=0.001)

In [ ]:
from antspynet.architectures import create_resnet_model_3d
from tensorflow.keras import Sequential, layers, models


def set_nontrainable_layers(model):
    model.trainable = False 
    return model

IMG_SHAPE = (172, 220, 156, 1)
N_CLASSES = 2

x = tf.keras.layers.Input(IMG_SHAPE)
base_model = create_resnet_model_3d(IMG_SHAPE, number_of_classification_labels=N_CLASSES)
base_model = set_nontrainable_layers(base_model)
fc_layer = tf.keras.layers.Dense(1024, activation='relu')(base_model.layers[-2].output)
#fc_layer = tf.keras.layers.Dropout(0.8)(fc_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=tf.keras.initializers.Constant(initial_bias))(fc_layer)
model = tf.keras.models.Model(inputs=base_model.input, outputs=output_layer)
model.summary()
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=METRICS)

In [19]:
EPOCHS = 20
BATCH_SIZE = 4
STEPS_PER_EPOCH = int(n_training_samples / BATCH_SIZE)
VALIDATION_STEPS = int(n_val_samples / BATCH_SIZE)

train_tensors = read_dataset(EPOCHS, BATCH_SIZE, train_tfrec)
val_tensors = read_dataset(EPOCHS, BATCH_SIZE, val_tfrec)
test_tensors = read_dataset_for_test(EPOCHS, BATCH_SIZE, test_tfrec)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(train_tensors,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=(val_tensors),
                    validation_steps=VALIDATION_STEPS,
                    shuffle=True,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[es],
                    verbose=1)

In [ ]:
def plot_loss_accuracy(history):

    with plt.style.context('seaborn-deep'):

        fig, ax = plt.subplots(1, 2, figsize=(15, 4))

        ## Plot Losses and Accuracies
        x_axis = np.arange(len(history.history['loss']))

        ax[0].set_title("Loss")
        ax[0].plot(x_axis, history.history['loss'], color="blue", linestyle=":", marker="X", label="Train Loss")
        ax[0].plot(x_axis, history.history['val_loss'], color="orange", linestyle="-", marker="X", label="Val Loss")

        ax[1].set_title("Accuracy")
        ax[1].plot(x_axis, history.history['accuracy'], color="blue", linestyle=":", marker="X", label="Train Accuracy")
        ax[1].plot(x_axis,
                   history.history['val_accuracy'],
                   color="orange",
                   linestyle="-",
                   marker="X",
                   label="Val Accuracy")

        ## Customization
        ax[0].grid(axis="x", linewidth=0.5)
        ax[0].grid(axis="y", linewidth=0.5)
        ax[0].legend()
        ax[1].grid(axis="x", linewidth=0.5)
        ax[1].grid(axis="y", linewidth=0.5)
        ax[1].legend()

        plt.show()

plot_loss_accuracy(history)

In [ ]:
res = model.evaluate(test_tensors)[1]
res

In [20]:
from antspynet.architectures import create_resnet_model_2d
from tensorflow.keras import Sequential, layers, models
from tensorflow.keras import optimizers

def set_nontrainable_layers(model):
    model.trainable = False 
    return model

IMG_SHAPE = (172, 220, 156)
N_CLASSES = 2

base_model2 = create_resnet_model_2d(IMG_SHAPE, number_of_classification_labels=N_CLASSES)
base_model2 = set_nontrainable_layers(base_model2)
flatten_layer2 = layers.Flatten()
dense_layer2 = layers.Dense(1024, activation='relu')
prediction_layer2 = layers.Dense(1, activation='sigmoid', bias_initializer=tf.keras.initializers.Constant(initial_bias))

model2 = models.Sequential([
        base_model2,
        flatten_layer2,
        dense_layer2,
        prediction_layer2])

model2.summary()

model2.compile(loss="binary_crossentropy", optimizer=opt, metrics=METRICS)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2)                 67420482  
                                                                 
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense_1 (Dense)             (None, 1024)              3072      
                                                                 
 dense_2 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 67,424,579
Trainable params: 4,097
Non-trainable params: 67,420,482
_________________________________________________________________


In [21]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=15, restore_best_weights=True)

history2 = model2.fit(train_tensors,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=(val_tensors),
                    validation_steps=VALIDATION_STEPS,
                    shuffle=True,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[es],
                    verbose=1)

Epoch 1/20


2022-11-29 20:44:21.632358: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600
2022-11-29 20:44:52.667046: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-29 20:44:53.042371: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.09GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-29 20:44:53.044279: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.09GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-29 20:44:53.519560: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate

  1/307 [..............................] - ETA: 3:38:16 - loss: 0.5745 - auc: 0.5000 - prc: 0.2500 - accuracy: 0.7500

: 

: 

In [ ]:
plot_loss_accuracy(history2)

In [ ]:
res2 = model2.evaluate(test_tensors)[1]
res2